In [1]:
import pandas as pd
import numpy as np
import os
import json
import pprint
from collections import Counter
import statsmodels.formula.api as smf
import os
import json
from pprint import pprint
from collections import Counter
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm

In [2]:
clean_data = pd.read_excel()

TypeError: read_excel() takes at least 1 argument (0 given)

# Actors

# Directors

# Gengr